In [7]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np

In [8]:
client = MongoClient("mongodb://localhost:27017/")

In [9]:
db = client.companies

In [10]:
companies = db.companies.find()

In [11]:
data_companies = pd.DataFrame(companies)

VISUALIZAMOS EL DATASET

In [12]:
data_companies['category_code'].value_counts()

web                 3787
software            2736
games_video         1083
mobile              1018
other                986
advertising          928
enterprise           742
ecommerce            688
consulting           637
network_hosting      626
public_relations     533
search               394
biotech              373
hardware             368
cleantech            305
semiconductor        167
security             156
analytics             66
finance               49
social                49
news                  48
education             36
music                 33
messaging             30
medical               25
travel                25
legal                 25
photo_video           23
health                23
manufacturing         19
sports                13
real_estate           10
fashion               10
automotive             9
hospitality            8
transportation         7
nanotech               5
nonprofit              4
design                 4
local                  1


SACAMOS QUERY

In [13]:
gaming = db.companies.find({"category_code":'games_video', "founded_year": {"$gt":1990}, "number_of_employees": {"$gt": 25}},
 {"name":1, "founded_year":1, "offices":1, "category_code":1, "total_money_raised":1, "number_of_employees":1,'description':1})

In [14]:
office_companies = pd.DataFrame(gaming)
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S..."
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd..."
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr..."
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ..."
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway..."


IMPORTAMOS FUNCIONES PARA SACAR LONGITUD Y LATITUD

In [15]:
#import coordenadas
def get_lon(x): 
    try: 
        return float(x['offices'][0]['longitude'])
    except: 
        return None
    
office_companies['longitud'] = office_companies.apply(lambda x: get_lon(x), axis=1)
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893


In [16]:
#import coordenadas
def get_lat(x): 
    try: 
        return float(x['offices'][0]['latitude'])
    except: 
        return None

In [17]:
office_companies['latitude'] = office_companies.apply(lambda x: get_lat(x), axis=1)
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619


In [18]:
office_companies.dropna(subset=['latitude'], inplace=True)
office_companies.dropna(subset=['longitud'], inplace=True)

In [19]:
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619


IMPORTAMOS FUNCIÓN GEO PARA SACAR EL GEO INDICE

In [20]:
#import geo
def createGeoJson(longitude,latitude):
    return {
        "type":"Point",
        "coordinates":[longitude,latitude]
    }

In [21]:
df = pd.DataFrame(office_companies)
df['position'] = df.apply(lambda x: createGeoJson(x["longitud"],x["latitude"]), axis=1)
display(df.head())
df.shape

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482,"{'type': 'Point', 'coordinates': [-122.409173,..."
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,..."
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936,"{'type': 'Point', 'coordinates': [-122.07948, ..."
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,..."


(71, 11)

In [22]:
df = df.drop('_id', axis=1)

In [23]:
df.head()

,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482,"{'type': 'Point', 'coordinates': [-122.409173,..."
1,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,..."
2,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936,"{'type': 'Point', 'coordinates': [-122.07948, ..."
3,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,..."


EXPORTAMOS DATASET MODIFICADO A JSON

In [ ]:
df.to_json(r'./offices2.json', orient="records")

In [ ]:
#mongoimport --db companies --collection limpiado --jsonArray offices.json ---> para importar el archivo json a Mongo
